# AdaptiveMD

## Example 2 - Running of Tasks

### 0. Imports

In [1]:
import sys, os

In [2]:
from adaptivemd import Project, Event, FunctionalEvent, Trajectory

Let's open our `test` project by its name. If you completed the previous example this should all work out of the box.

In [3]:
project = Project('tutorial')

Open all connections to the `MongoDB` and `Session` so we can get started.

Let's see where we are. These numbers will depend on whether you run this notebook for the first time or just continue again. Unless you delete your project it will accumulate models and files over time, as is our ultimate goal.

In [4]:
print project.tasks

print project.trajectories
print project.models

<StoredBundle for with 2 file(s) @ 0x1131d5550>
<ViewBundle for with 1 file(s) @ 0x1131d5690>
<StoredBundle for with 0 file(s) @ 0x1131d5490>


Now restore our old ways to generate tasks by loading the previously used generators.

In [5]:
engine = project.generators['openmm']
modeller = project.generators['pyemma']
pdb_file = project.files['initial_pdb']

Remember that we stored some files in the database and of course you can look at them again, should that be important.

In [6]:
print pdb_file.get_file()[:1000] + ' [...]'

REMARK   1 CREATED WITH MDTraj 1.8.0, 2016-12-22
CRYST1   26.063   26.063   26.063  90.00  90.00  90.00 P 1           1 
MODEL        0
ATOM      1  H1  ACE A   1      -1.900   1.555  26.235  1.00  0.00          H   
ATOM      2  CH3 ACE A   1      -1.101   2.011  25.651  1.00  0.00          C   
ATOM      3  H2  ACE A   1      -0.850   2.954  26.137  1.00  0.00          H   
ATOM      4  H3  ACE A   1      -1.365   2.132  24.600  1.00  0.00          H   
ATOM      5  C   ACE A   1       0.182   1.186  25.767  1.00  0.00          C   
ATOM      6  O   ACE A   1       1.089   1.407  26.645  1.00  0.00          O   
ATOM      7  N   ALA A   2       0.302   0.256  24.807  1.00  0.00          N   
ATOM      8  H   ALA A   2      -0.588   0.102  24.354  1.00  0.00          H   
ATOM      9  CA  ALA A   2       1.498  -0.651  24.567  1.00  0.00          C   
ATOM     10  HA  ALA A   2       1.810  -0.944  25.570  1.00  0.00          H   
ATOM     11  CB  ALA A   2       1.054  -1.959  23.852

### 1. Run simulations

Before we talk about adaptivity, let's have a look at possibilities to generate trajectories.

We assume that you successfully ran a first trajectory using a worker. Next, we talk about lot's of ways to generate new trajectories.

#### Trajectories from a pdb

You will do this in the beginning. Remember we already have a PDB stored from setting up the engine. if you want to start from this configuration do as before

1. get the trajectory you want
2. make a task
3. submit the task

##### The `Trajectory` object

In [7]:
file_name = next(project.traj_name)

trajectory = Trajectory(
    location=file_name,                          # this creates a new filename
    frame=pdb_file,                              # initial frame is the PDB
    length=100,                                  # length is 100 frames
    engine=engine                                # the engine to be used
)

Since this is tedious to write there is a shortcut

In [8]:
trajectory = project.new_trajectory(
    frame=pdb_file,
    length=100,
    engine=engine,
    number=1          # if more then one you get a list of trajectories
)

Compute the least common multiple of all strides. You can only simulate multiples of this number. Otherwise you will not be able to extend a trajectory since the reporter will write in not equal distances.

In [9]:
engine.native_stride

10

In [11]:
trajectory.short

'sandbox:///{}/00000003/'

##### The `Task` object

In [12]:
task_run = engine.run(trajectory)

This was easy, but we can do some interesting stuff. Since we know the trajectory will exist now we can also extend by some frames.

In [13]:
task_extend = engine.extend(trajectory, 50)

The only problem is to make sure the tasks are run in the correct order. This would not be a problem if the worker will run tasks in the order they are place in the queue, but that defeats the purpose of parallel runs. Therefore an extended tasks knows that is depends on the existance of the source trajectory. The worker will hence only run a trajectory, once the source exists.

In [14]:
project.queue(task_run, task_extend)

There is also a shorter way of writing this

In [15]:
# task = trajectory.run().extend(50)

This will create two tasks that first runs the trajectory and then extend it by 50 frames (in native engine frames)

#### Check the results

For a seconds let's see if everything went fine.

In [16]:
for t in project.trajectories:
    print t.short, t.length

sandbox:///{}/00000000/ 100
sandbox:///{}/00000001/ 100
sandbox:///{}/00000003/ 150


If this works, then you should see one 100 frame trajectory from the setup (first example) and a second 150 length trajectory that we just generated by running 100 frames and extending it by another 50.

If not, there might be a problem or (more likely) the tasks are not finished yet. Just try the above cell again and see if it changes to the expected output.

Let's do something stupid and produce an error by using a wrong initial pdb file.

In [32]:
trajectory = project.new_trajectory(engine['system_file'], 100)
task = engine.run(trajectory)
project.queue(task)

Well, nothing changed obviously and we expect it to fail. So let's inspect what happened.

In [39]:
task.state

u'fail'

It failed, well, we kind of knew that. No suprise here, but why? Let's look at the stdout and stderr

In [40]:
print task.stdout

13:01:09 [worker:3] stdout from running task
GO...
Reading PDB



In [41]:
print task.stderr

13:01:09 [worker:3] stderr from running task
Traceback (most recent call last):
  File "openmmrun.py", line 142, in <module>
    pdb = PDBFile(args.topology_pdb)
  File "/Users/jan-hendrikprinz/anaconda/lib/python2.7/site-packages/simtk/openmm/app/pdbfile.py", line 159, in __init__
    self.positions = self._positions[0]
IndexError: list index out of range



We see, what we expect. In `openmmrun.py` the openmm executable it could not load the pdb file. 

> *NOTE* If your worker dies for some reason, it will not set a STDOUT or STDERR. If you think that your task should be able to execute, then you can do `task.state = 'created'` and reset it to be accessible to workers. This is NOT recommended, just to explain how this works. Of course you need a new worker anyway.

#### What else

If you have the trajectory object and want to just create the trajectory, you can also put this in the queue. A `Task` object is generated for you, BUT you will have no direct access to the task as in this example. Do whatever you need

In [17]:
project.queue(project.new_trajectory(pdb_file, 100).run())

### Trajectories from other trajectories

This will be the most common case. At least in any remote kind of adaptivity you will not start always from the same position or extend. You want to pick any exisiting frame and continue from there. So, let's do that.

First we get a trajectory. Every bundle in the project (e.g. `.trajectories`, `.models`, `.files`, `.tasks`) acts like an enhanced set. You can iterate over all entries as we did before, and you can get one element, which usually is the first stored, but not always. For now that is enough

In [18]:
trajectory = project.trajectories.one

Good, at least 100 frames. We pick, say, frame at index 28 (which is the 29th frame, we start counting at zero) using the way you pick an element from a python list (which is almost what a `Trajectory` represents, a list of frames)

In [19]:
frame = trajectory[28]
print frame, frame.exists

Frame(sandbox:///{}/00000000/[28]) False


In [20]:
frame = trajectory[30]
print frame, frame.exists

Frame(sandbox:///{}/00000000/[30]) True


This part is important! We are running only one full atom trajectory with stride, so if we want to pick a frame from this trajectory you can pick in theory every frame, but only some of these really exist. If you want to restart from a frame this needs to be the case. Otherwise you run into trouble.

To run a trajectory just use the frame as the initial frame.

In [21]:
frame = trajectory[28]
task = project.new_trajectory(frame, 100, engine).run()
print task

None


In [22]:
frame = trajectory[30]
task = project.new_trajectory(frame, 100, engine).run()
print task

In [23]:
print task.description

Task: TrajectoryGenerationTask(OpenMMEngine) [created]

Sources
- staging:///integrator.xml 
- sandbox:///{}/00000000/ [exists]
- staging:///alanine.pdb 
- staging:///openmmrun.py 
- staging:///system.xml 
Targets
- sandbox:///{}/00000006/
Modified

<pretask>
Link('staging:///alanine.pdb' > 'worker://initial.pdb)
Link('staging:///system.xml' > 'worker://system.xml)
Link('staging:///integrator.xml' > 'worker://integrator.xml)
Link('staging:///openmmrun.py' > 'worker://openmmrun.py)
Link('sandbox:///{}/00000000/' > 'worker://source/)
mdconvert -o worker://input.pdb -i 3 -t worker://initial.pdb worker://source/master.dcd
Touch('worker://traj/')
python openmmrun.py -r --report-interval 1 -p CPU --types="{'protein':{'stride':1,'selection':'protein','filename':'protein.dcd'},'master':{'stride':10,'selection':null,'filename':'master.dcd'}}" -t worker://input.pdb --length 100 worker://traj/
Move('worker://traj/' > 'sandbox:///{}/00000006/)
<posttask>


See, how the actual frame picked in the `mdconvert` line is `-i 3` meaning index 3 which represents frame 30 with stride 10.

Now, run the task.

In [24]:
project.queue(task)

Btw, you can wait until something happens using `project.wait_until(condition)`. This is not so useful in notebooks, but in scripts it does. `condition` here is a function that evaluates to `True` or `False`. it will be tested in regular intervals and once it is `True` the function returns.

In [25]:
project.wait_until(task.is_done)

In [26]:
task.state

u'success'

Each `Task` has a function `is_done` that you can use. It will return once a task is done. That means it either failed or succeeded or was cancelled. Basically when it is not queued anymore.

If you want to run adaptively, _all you need to do_ is to figure out where to start new simulations from and use the methods provided to run these.

### `Model` tasks

A model generating task work similar to trajectories. You create the generator with options (so far, this will become more complex in the future) and then you create a `Task` from passing it a list of trajectories to be analyzed.

In [7]:
task = modeller.execute(list(project.trajectories))
project.queue(task)

In [16]:
print project.models

<StoredBundle for with 1 file(s) @ 0x1131d5490>


In [17]:
m = project.models.last

In [24]:
m['input']['modeller'] is modeller

True

In [20]:
project.storage.models._document.find_one()

{u'_cls': u'Model',
 u'_dict': {u'data': {u'_hex_uuid': u'0x4f26c7110fb311e79b82000000000112L',
   u'_store': u'data'}},
 u'_id': u'4f26c711-0fb3-11e7-9b82-000000000114',
 u'_obj_uuid': u'4f26c711-0fb3-11e7-9b82-000000000114',
 u'_time': 1490264885}

In [30]:
print project.storage.data.save(project.models.one)

173367363512597802520375418252368543750


In [35]:
for f in project.files:
    if 'output' in f.path:
        print f, f.get_file()

file:///Users/jan-hendrikprinz/Studium/git/adaptivemd/examples/tutorial/_rpc_output_0x3d54b3630fae11e7acad000000000114L.json {"_cls":"Model","_dict":{"data":{"clustering":{"dtrajs":[{"_data":"AgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAADAAAAAwAAAAMAAAADAAAAAwAAAAMAAAADAAAAAwAAAAMAAAADAAAAAwAAAAMAAAADAAAAAwAAAAMAAAADAAAAAwAAAAMAAAADAAAAAwAAAAMAAAADAAAAAwAAAAMAAAADAAAAAwAAAAMAAAADAAAAAwAAAAMAAAADAAAAAwAAAAMAAAADAAAAAwAAAAMAAAADAAAAAwAAAAMAAAADAAAAAwAAAAMAAAADAAAAAwAAAAMAAAA=","_numpy":{"_tuple":[101]},"_dtype":"int32"},{"_data":"AgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAEAAAABAAAAAQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [38]:
task.output.data

{'clustering': {'dtrajs': [array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
          2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
          3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
          3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32),
   array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
          3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32),
   array([2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
          2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3,
          3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
   

In [31]:
print project.data

<StoredBundle for with 1 file(s) @ 0x10a4cf450>


In [86]:
project.wait_until(task.is_done)

In [100]:
project.models.last.data

{'clustering': {'dtrajs': [array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
          4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4,
          4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2,
          2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 4,
          4, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
   array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3,
          3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
          3, 3, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
   array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
   

In [13]:
for m in project.models:
    print m

So we generated one model. The `Model` objects contain (in the base version) only a `.data` attribute which is a dictionary of information about the generated model.

In [101]:
model = project.models.last
model.data.keys()

['clustering', 'input', 'msm', 'features', 'tica']

In [102]:
print model.data['msm']['P']

[[  9.29867992e-01   2.31225753e-02   0.00000000e+00   2.56911421e-02
    2.13182910e-02]
 [  5.02943610e-02   9.32885914e-01   1.58694970e-02   0.00000000e+00
    9.50227702e-04]
 [  0.00000000e+00   8.41927902e-03   9.30501922e-01   0.00000000e+00
    6.10787985e-02]
 [  3.58491427e-02   0.00000000e+00   0.00000000e+00   9.20768307e-01
    4.33825503e-02]
 [  2.38045346e-02   4.87811353e-04   5.91021067e-02   3.47157879e-02
    8.81889759e-01]]


### Pick frames automatically

The last thing that is implemented is a function that can utilize models to decide which frames are better to start from. The simplest one will use the counts per state, take the inverse and use this as a distribution.

In [29]:
for m in project.models:
    d = m.data
    p = [f.path for f in d['input']['trajectories']]
    print len(p), len(set(p)), len(d['input']['lengths'])
    print zip([len(f) for f in d['input']['trajectories']], d['input']['lengths'])

35 34 34
[(100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 151), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (150, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101)]
36 35 35
[(100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 151), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (150, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101)]
37 36 36
[(100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 101), (100, 151), (100, 101), (100, 101), (100, 101), (100, 101)

In [26]:
zip([(len(f), f.short) for f in m['input']['trajectories']], [f for f in m['input']['lengths']])

[((100, 'sandbox:///{}/00000016/'), 101),
 ((100, 'sandbox:///{}/00000017/'), 101),
 ((100, 'sandbox:///{}/00000018/'), 101),
 ((100, 'sandbox:///{}/00000019/'), 101),
 ((100, 'sandbox:///{}/00000020/'), 101),
 ((100, 'sandbox:///{}/00000021/'), 101),
 ((100, 'sandbox:///{}/00000022/'), 101),
 ((100, 'sandbox:///{}/00000023/'), 101),
 ((100, 'sandbox:///{}/00000024/'), 151),
 ((100, 'sandbox:///{}/00000025/'), 101),
 ((100, 'sandbox:///{}/00000026/'), 101),
 ((100, 'sandbox:///{}/00000027/'), 101),
 ((100, 'sandbox:///{}/00000028/'), 101),
 ((100, 'sandbox:///{}/00000029/'), 101),
 ((100, 'sandbox:///{}/00000030/'), 101),
 ((100, 'sandbox:///{}/00000031/'), 101),
 ((100, 'sandbox:///{}/00000032/'), 101),
 ((100, 'sandbox:///{}/00000033/'), 101),
 ((100, 'sandbox:///{}/00000034/'), 101),
 ((100, 'sandbox:///{}/00000035/'), 101),
 ((100, 'sandbox:///{}/00000036/'), 101),
 ((100, 'sandbox:///{}/00000037/'), 101),
 ((100, 'sandbox:///{}/00000038/'), 101),
 ((100, 'sandbox:///{}/00000039/')

array([101, 101, 101, 101, 101, 101, 101, 101, 151, 101, 101, 101, 101,
       101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
       101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
       101, 101, 101, 101])

In [13]:
project.find_ml_next_frame(40)

[(39, 90), (1, 50), (18, 80), (35, 10), (6, 10), (13, 60), (8, 110), (6, 80), (21, 90), (35, 40), (42, 10), (32, 90), (18, 0), (9, 40), (30, 0), (11, 50), (29, 40), (7, 60), (25, 80), (23, 90), (35, 70), (16, 70), (17, 60), (11, 20), (0, 20), (36, 80), (4, 60), (42, 30), (8, 140), (4, 0), (12, 50), (41, 30), (7, 20), (25, 70), (27, 80), (18, 80), (30, 40), (35, 60), (8, 80), (23, 30)]


[Frame([90]),
 Frame([50]),
 Frame([80]),
 Frame([10]),
 Frame([10]),
 Frame([60]),
 None,
 Frame([80]),
 Frame([90]),
 Frame([40]),
 Frame([10]),
 Frame([90]),
 Frame([0]),
 Frame([40]),
 Frame([0]),
 Frame([50]),
 Frame([40]),
 Frame([60]),
 Frame([80]),
 Frame([90]),
 Frame([70]),
 Frame([70]),
 Frame([60]),
 Frame([20]),
 Frame([20]),
 Frame([80]),
 Frame([60]),
 Frame([30]),
 None,
 Frame([0]),
 Frame([50]),
 Frame([30]),
 Frame([20]),
 Frame([70]),
 Frame([80]),
 Frame([80]),
 Frame([40]),
 Frame([60]),
 Frame([80]),
 Frame([30])]

So you can pick states according to the newest (last) model. (This will be moved to the Brain). And since we want trajectories with these frames as starting points there is also a function for that

In [73]:
trajectories = project.new_ml_trajectory(length=100, number=4, engine=engine)
trajectories

[Trajectory(Frame([100]) >> [0..100]),
 Trajectory(Frame([60]) >> [0..100]),
 Trajectory(Frame([60]) >> [0..100]),
 Trajectory(Frame([60]) >> [0..100])]

Let's submit these before we finish this notebook with a quick discussion of workers

In [74]:
project.queue(trajectories)

That's it.

### What about workers

Worker are the instances that execute tasks for you. If you did not stop the worker it will still be running and you can check its state

In [75]:
from adaptivemd import DT

DT is a little helper to convert time stamps into something readable.

In [76]:
project.trigger()
for w in project.workers:
    if w.state == 'running':
        print '[%s:%s] %s:%s' % (w.state, DT(w.seen).time, w.hostname, w.cwd)

[running:13:49:47] Stevie.fritz.box:/Users/jan-hendrikprinz/Studium/git/adaptivemd


In [78]:
for t in project.tasks:
    print t.state

success
success
fail
fail
success
success
success
success
dummy
success
success
success
success
success


Okay, the worker is running, was last reporting its heartbeat at ... and has a hostname and current working directory (where it was executed from). The generators specify which tasks from some generators are executed. If it is `None` then the worker runs all tasks it finds. You can use this to run specific workers for models and some for trajectory generation.

You can also control it remotely by sending it a command. `shutdown` will shut it down for you.

In [36]:
# project.workers.last.command = 'shutdown'

Afterwards you need to restart you worker to continue.

In [79]:
project.close()